In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Fine-tuning multimodal RoBERTa + tabular para IDEOLOGY,
guarda checkpoints por fold, guarda el mejor modelo y
genera un JSON con los tweets que el modelo acierta con
alta confianza (≥ 0.80) incluyendo su traducción automática al español.
"""

# ──────────────────── IMPORTS ────────────────────
import pathlib, json
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

from googletrans import Translator  # pip install googletrans==4.0.0-rc1
from joblib import dump

tr = Translator()
# ──────────────────── CONFIG ─────────────────────
MODEL_NAME      = "cardiffnlp/twitter-roberta-base"
TARGET          = "age_group"
COHENS_D_FILE   = "../cohens_d_completo.csv"
INPUT_CSV       = "features_linguisticas_en_con_glove.csv"

BATCH_SIZE      = 16
EPOCHS          = 10
LR              = 1e-5
FOLDS           = 5
COHEN_THRESHOLD = 0.10
CONF_TH         = 0.70

ckpt_dir = pathlib.Path("checkpoints_age")
ckpt_dir.mkdir(exist_ok=True)
out_dir = pathlib.Path("outputs_age")
out_dir.mkdir(exist_ok=True)

# ──────────────────── DATA ───────────────────────
df = pd.read_csv(INPUT_CSV)
cohen_df = pd.read_csv(COHENS_D_FILE)

df = df[df[TARGET].isin(["mayor", "menor"])].copy()
df["label"] = df[TARGET].map({"menor": 0, "mayor": 1})

# ──────────────────── DATA ───────────────────────
df = pd.read_csv(INPUT_CSV)
cohen_df = pd.read_csv(COHENS_D_FILE)

df = df[df["ideology"].isin(["izquierda", "derecha"])].copy()
df["label"] = df["ideology"].map({"izquierda": 0, "derecha": 1})

selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) &
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD) &
    (~cohen_df["variable"].str.startswith("XWE-"))
]["variable"].unique().tolist()

print(f"🔎 Variables seleccionadas (|d| > {COHEN_THRESHOLD}): {selected_vars}")

text_col = "clean_text"
assert text_col in df.columns, f"{text_col} no está en el CSV"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

scaler = StandardScaler()
df[selected_vars] = scaler.fit_transform(df[selected_vars])

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ──────────────────── DATASET ────────────────────
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True, padding="max_length", max_length=128,
            return_tensors="pt"
        )
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float32)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# ──────────────────── MODELO ─────────────────────
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 64),
            nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_emb = out.last_hidden_state[:, 0]
        tab_emb = self.tabular_net(nums)
        combined = torch.cat([cls_emb, tab_emb], dim=1)
        return self.classifier(combined)

# ──────────────────── TRAIN / EVAL ───────────────
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_w = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_w, dtype=torch.float32).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
f1_scores = []
selected = []

# inicializar mejor modelo
best_f1 = 0.0
best_state = None
best_fold = None

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels), 1):
    print(f"\n📦 Fold {fold}/{FOLDS}")
    train_ds = MultiModalDataset([texts[i] for i in train_idx], features[train_idx], labels[train_idx])
    val_ds = MultiModalDataset([texts[i] for i in val_idx], features[val_idx], labels[val_idx])
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    model.train()
    for epoch in range(1, EPOCHS + 1):
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold} · Epoch {epoch}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            logits = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(logits, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch} loss: {total_loss/len(train_dl):.4f}")

    model.eval()
    all_preds, all_lbls = [], []

    with torch.no_grad():
        for b, batch in enumerate(val_dl):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            logits = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            probs = torch.softmax(logits, dim=1)
            confs, preds = probs.max(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_lbls.extend(lbls.cpu().numpy())

            for j in range(len(preds)):
                if preds[j] == lbls[j] and confs[j] >= CONF_TH:
                    idx_global = val_idx[b * BATCH_SIZE + j]
                    tweet_en = texts[idx_global]
                    try:
                        tweet_es = tr.translate(tweet_en, dest="es").text
                    except Exception:
                        tweet_es = ""

                    selected.append({
                        "row": int(idx_global),
                        "text_en": tweet_en,
                        "text_es": tweet_es,
                        "prob": round(float(confs[j]), 4),
                        "pred": int(preds[j]),
                        "true": int(lbls[j]),
                        "fold": fold
                    })

    f1 = f1_score(all_lbls, all_preds, average="macro")
    f1_scores.append(f1)
    print(f"✅ F1 Fold {fold}: {f1:.4f}")
    print(classification_report(all_lbls, all_preds, target_names=["izquierda", "derecha"]))

    if f1 > best_f1:
        best_f1 = f1
        best_fold = fold
        best_state = {
            "model_state": model.state_dict(),
            "backbone": MODEL_NAME,
            "selected_vars": selected_vars
        }

    # checkpoint parcial
    fold_ckpt = ckpt_dir / f"fold{fold}"
    fold_ckpt.mkdir(exist_ok=True)
    model.transformer.save_pretrained(fold_ckpt)
    tokenizer.save_pretrained(fold_ckpt)

# ──────────────────── RESULTADOS FINALES ─────────
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {np.mean(f1_scores):.4f}")

json_path = out_dir / "clear_tweets_ideology.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(selected, f, ensure_ascii=False, indent=2)
print(f"📝 Guardados {len(selected)} tweets claros en {json_path.resolve()}")

# Guardar mejor modelo
if best_state is not None:
    torch.save(best_state, out_dir / "best_model.pt")
    print(f"🔒 Guardado modelo completo en best_model.pt (fold {best_fold}, F1={best_f1:.4f})")

# Guardar scaler
dump(scaler, out_dir / "scaler.pkl")
print("💾 Guardado StandardScaler en scaler.pkl")


🔎 Variables seleccionadas (|d| > 0.1): ['Xwords', 'Xunique', 'Xmax_length', 'Xlength_3', 'Xstop', 'Xcharacter', 'Xcapital', 'Xpunctuation', 'Xword_par', 'Xchar_par', 'Xdet', 'Xprep', 'Xsing', 'Xplural', 'Xadv', 'Xnouns', 'Xconj', 'Xpast', 'Xhashtag', 'Xurl']

📦 Fold 1/5


Fold 1 · Epoch 1: 100%|██████████| 48/48 [00:04<00:00, 10.43it/s]


🔁 Epoch 1 loss: 0.6918


Fold 1 · Epoch 2: 100%|██████████| 48/48 [00:04<00:00, 10.47it/s]


🔁 Epoch 2 loss: 0.6702


...

🔁 Epoch 8 loss: 0.1150


Fold 5 · Epoch 9: 100%|██████████| 48/48 [00:04<00:00, 10.39it/s]


🔁 Epoch 9 loss: 0.1215


Fold 5 · Epoch 10: 100%|██████████| 48/48 [00:04<00:00, 10.35it/s]
/tmp/ipykernel_3276282/2556984385.py:189: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  tweet_es = ""


🔁 Epoch 10 loss: 0.0635
✅ F1 Fold 5: 0.6790
              precision    recall  f1-score   support

   izquierda       0.76      0.80      0.78       122
     derecha       0.61      0.56      0.58        70

    accuracy                           0.71       192
   macro avg       0.68      0.68      0.68       192
weighted avg       0.70      0.71      0.71       192


📊 F1 macro por fold: [0.6631, 0.7172, 0.6511, 0.638, 0.679]
🏁 F1 macro promedio final: 0.6697
📝 Guardados 628 tweets claros en /home/jupyter-lquijano/marcos/ingleses/outputs/clear_tweets_ideology.json
🔒 Guardado modelo completo en best_model.pt (fold 2, F1=0.7172)
💾 Guardado StandardScaler en scaler.pkl


In [4]:
# ──────────────────── RESULTADOS FINALES ─────────
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {np.mean(f1_scores):.4f}")
out_dir    = pathlib.Path("outputs")
# ----------- JSON con tweets claros --------------
json_path = out_dir / "clear_tweets_ideology.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(selected, f, ensure_ascii=False, indent=2)

print(f"📝 Guardados {len(selected)} tweets claros en {json_path.resolve()}")

# ----------- Guardar mejor modelo ---------------
# Guardamos el mejor fold completo: arquitectura + pesos + vars
torch.save(best_state, out_dir / "best_model.pt")
print(f"🔒 Guardado modelo completo en best_model.pt (fold {best_fold}, F1={best_f1:.4f})")

# ----------- Guardar scaler ---------------------
from joblib import dump
dump(scaler, out_dir / "scaler.pkl")
print("💾 Guardado StandardScaler en scaler.pkl")



📊 F1 macro por fold: [0.6631, 0.7172, 0.6511, 0.638, 0.679]
🏁 F1 macro promedio final: 0.6697
📝 Guardados 628 tweets claros en /home/jupyter-lquijano/marcos/ingleses/outputs/clear_tweets_ideology.json
🔒 Guardado modelo completo en best_model.pt (fold 2, F1=0.7172)
💾 Guardado StandardScaler en scaler.pkl
